# MySQL + SPARK

![my-saprk](spark.png)

El cuaderno ejemplifica la conección de spark con una base de datos relacional en este caso MySQL

### Lectura de una tabla de mysql

## Las fuentes de información que se ocupan en el ejercicio son datos de un concurso de kaggle sobre BIMBO

In [ ]:
%%bash

jps


#### Conexion Spark BdRelacional

In [ ]:
sc

In [ ]:
from pyspark.sql import Row

In [ ]:
df_ss = spark.read.format("jdbc").options(
    url="jdbc:mysql://localhost/bimbodb",
    driver="com.mysql.jdbc.Driver",
    dbtable="sample_submission",
    user="saxsauser",
    password="ggsaXsa").load()

In [ ]:
df_ss.describe().show()

In [ ]:
df_ss.printSchema()

In [ ]:
df_ss.createOrReplaceTempView('sample_submission')

In [ ]:
%%sql
SELECT Demanda_uni_equil, COUNT(Demanda_uni_equil) AS Apariciones FROM sample_submission GROUP BY Demanda_uni_equil

In [ ]:
df_t = spark.read.format("jdbc").options(
    url="jdbc:mysql://localhost/bimbodb",
    driver="com.mysql.jdbc.Driver",
    dbtable="test",
    user="saxsauser",
    password="ggsaXsa").load()

In [ ]:
column = df_t.columns
column

In [ ]:
for col in column:
    df_t.select(col).describe().show()

In [ ]:
df_sel = df_t.select('Agencia_ID', 'Cliente_ID', 'Producto_ID')
df_sel.show(20)

In [ ]:
df_sel.select('Producto_ID').distinct().count()

In [ ]:
df_wri = df_sel.groupby('Producto_ID').count().sort('count', ascending=False)
df_wri.show()

In [ ]:
df_wri.write.format("com.mongodb.spark.sql.DefaultSource")\
    .mode("append")\
    .option("spark.mongodb.input.uri", "mongodb://127.0.0.1/bimbodb.mysqltomongo?readPreference=primaryPreferred")\
    .option("spark.mongodb.output.uri", "mongodb://127.0.0.1/bimbodb.mysqltomongo").save()